In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [2]:
# data prep from previous module
file=r'census_income.csv'

ci=pd.read_csv(file)
ci.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,Y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# there is perfect correspondance between education and education.num, we'll drop education
ci.drop('education',axis=1,inplace=True)

In [4]:
# convert target Y to 1,0
ci['Y']=(ci['Y']==' >50K').astype(int)

In [5]:
cat_cols=ci.select_dtypes(['object']).columns 

In [6]:
cat_cols

Index(['workclass', 'marital.status', 'occupation', 'relationship', 'race',
       'sex', 'native.country'],
      dtype='object')

## convert to dummies only those categories for each categorical variable for whicch frequenccy is more than 100

In [7]:
ci.workclass.value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [14]:
for col in cat_cols:
    freqs=ci[col].value_counts()
    selected_cats=freqs.index[freqs>100][:-1]
    
    print(col)
    for cat in selected_cats:
        name=col+'_'+cat
        ci[name]=(ci[col]==cat).astype(int)
    del ci[col]

workclass
marital.status
occupation
relationship
race
sex
native.country


In [18]:
ci.columns = ci.columns.str.replace(' ', '')

In [19]:
x_train=ci.drop('Y',axis=1)
y_train=ci['Y']

In [20]:
gbm_params={'n_estimators':[50,100,200,500,700],
           'learning_rate': [0.01,.05,0.1,0.4,0.8,1],
            'max_depth':[1,2,3,4,5,6],
            #'min_samples_split':[2,5,10,20],
            #'min_samples_leaf':[2,5,10,20],
            'subsample':[0.5,0.8,1],
            'max_features':[5,10,15,20,30,45]
           }

In [21]:
gbm=GradientBoostingClassifier()

In [43]:
random_search=RandomizedSearchCV(gbm,scoring='roc_auc',param_distributions=gbm_params,
                                 cv=5,n_iter=10,n_jobs=-1)

In [26]:
random_search.fit(x_train,y_train)
# this might take upto 30-45 miins to finish , if you try cv=10 and larger number for n_iter
# ( dont be impatient :) . Also dont be alarmed if it finishes early :))

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [50, 100, 200, 500, 700], 'learning_rate': [0.01, 0.05, 0.1, 0.4, 0.8, 1], 'max_depth': [1, 2, 3, 4, 5, 6], 'subsample': [0.5, 0.8, 1], 'max_features': [5, 10, 15, 20, 30, 45]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.4, loss='deviance', max_depth=3,
              max_features=20, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=0.8, verbose=0, warm_start=False)
              
use the above result in the class, its a result from previous run. This can be definitely different on a rerun. use this to save time in class so that you dont have to wait for the randomised search to finish

In [27]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [28]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.920 (std: 0.00229)
Parameters: {'subsample': 1, 'n_estimators': 100, 'max_features': 10, 'max_depth': 2, 'learning_rate': 0.4}

Model with rank: 2
Mean validation score: 0.915 (std: 0.00375)
Parameters: {'subsample': 0.8, 'n_estimators': 100, 'max_features': 5, 'max_depth': 1, 'learning_rate': 0.4}

Model with rank: 3
Mean validation score: 0.914 (std: 0.00291)
Parameters: {'subsample': 0.5, 'n_estimators': 100, 'max_features': 30, 'max_depth': 2, 'learning_rate': 0.1}

Model with rank: 4
Mean validation score: 0.914 (std: 0.00267)
Parameters: {'subsample': 0.5, 'n_estimators': 100, 'max_features': 20, 'max_depth': 2, 'learning_rate': 0.1}

Model with rank: 5
Mean validation score: 0.913 (std: 0.00154)
Parameters: {'subsample': 0.8, 'n_estimators': 700, 'max_features': 20, 'max_depth': 2, 'learning_rate': 1}



top 5 classfiers from the previous run were as follows : 

Model with rank: 1

Mean validation score: 0.925 (std: 0.00188)

Parameters: {'max_features': 20, 'max_depth': 3, 'subsample': 0.8, 'learning_rate': 0.4, 'n_estimators': 100}

~~~~~~~~~~

Model with rank: 2

Mean validation score: 0.924 (std: 0.00121)

Parameters: {'max_features': 15, 'max_depth': 4, 'subsample': 1, 'learning_rate': 0.4, 'n_estimators': 100}

~~~~~~~~~~

Model with rank: 3

Mean validation score: 0.923 (std: 0.00250)

Parameters: {'max_features': 5, 'max_depth': 4, 'subsample': 0.5, 'learning_rate': 0.05, 'n_estimators': 500}

~~~~~~~~~~

Model with rank: 4

Mean validation score: 0.914 (std: 0.00290)

Parameters: {'max_features': 10, 'max_depth': 5, 'subsample': 1, 'learning_rate': 0.05, 'n_estimators': 50}

~~~~~~~~~~

Model with rank: 5

Mean validation score: 0.913 (std: 0.00174)

Parameters: {'max_features': 30, 'max_depth': 5, 'subsample': 0.8, 'learning_rate': 0.4, 'n_estimators': 200}

tentative performance : 0.925 for the best classfier 

**Note: you can use the random search predict,predict_proba function to make prediction as randomisedsearchcv automatically fits the best candidate on complete data. If you want to look into feature_importance etc, then fit the best estimator separately**

In [31]:
?GradientBoostingClassifier

Init signature:
GradientBoostingClassifier(
    loss='deviance',
    learning_rate=0.1,
    n_estimators=100,
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    init=None,
    random_state=None,
    max_features=None,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    presort='auto',
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
)
Docstring:     
Gradient Boosting for classification.

GB builds an additive model in a
forward stage-wise fashion; it allows for the optimization of
arbitrary differentiable loss functions. In each stage ``n_classes_``
regression trees are fit on the negative gradient of the
binomial or multinomial deviance loss function. Binary classification
is a special case where only a single regression tree is induced.

Read more in the :ref:`User Guide <gradient_boosti

In [29]:
# Considering the best estimator
gbm = GradientBoostingClassifier(criterion='friedman_mse', init=None, learning_rate=0.4, loss='deviance', max_depth=3,
                                 max_features=20, max_leaf_nodes=None, min_impurity_split=1e-07, min_samples_leaf=1, 
                                 min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100, presort='auto', 
                                 random_state=None, subsample=0.8, verbose=0, warm_start=False)

In [32]:
gbm.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_spli

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.4, loss='deviance', max_depth=3,
              max_features=20, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [33]:
from sklearn.model_selection import cross_val_score

In [34]:
cross_val_score(gbm,x_train,y_train,scoring='roc_auc',verbose=10,n_jobs=-1,cv=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    9.9s remaining:    9.9s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   10.0s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.3s finished


array([0.92285908, 0.92259118, 0.92692209, 0.91751391, 0.92609703,
       0.92885264, 0.92396086, 0.92789446, 0.92880182, 0.92638676])

In [15]:
scores=[0.92426515, 0.92337548, 0.92699691, 0.91893157, 0.92382129,
       0.93086085, 0.92734984, 0.92621726, 0.92966944, 0.92405864]

In [16]:
import numpy as np

In [17]:
np.mean(scores)

0.9255546429999999

In [18]:
np.std(scores)

0.0032611116490302576